In [ ]:
import pandas as pd

df = pd.read_csv("../data/access_logs_cleaned.csv", parse_dates=["timestamp"])
df.head()

,user_id,timestamp,file_accessed,access_type,location,role,failed_logins,hour,day_of_week
0,1104,2024-01-01 08:06:00,Finance_Report,read,Office,Employee,0,8,Monday
1,1034,2024-01-01 09:05:00,Finance_Report,read,Office,Employee,1,9,Monday
2,1051,2024-01-01 09:05:00,HR_Records,read,Office,Employee,0,9,Monday
3,1103,2024-01-01 07:21:00,Client_Data,read,Remote,Admin,0,7,Monday
4,1039,2024-01-01 10:07:00,Engineering_Design,write,Remote,Employee,0,10,Monday


In [ ]:
df["anomaly_failed_login"] = df["failed_logins"] >= 5

In [ ]:
df["anomaly_off_hours"] = (df["hour"] < 8) | (df["hour"] > 20)

In [ ]:
access_counts = df.groupby("user_id").size()

threshold = access_counts.quantile(0.95)
high_activity_users = access_counts[access_counts > threshold].index

df["anomaly_excessive_access"] = df["user_id"].isin(high_activity_users)

In [ ]:
df["is_anomaly"] = (
    df["anomaly_failed_login"]
    | df["anomaly_off_hours"]
    | df["anomaly_excessive_access"]
)

df["is_anomaly"].value_counts()

is_anomaly
False    8501
True     6499
Name: count, dtype: int64

In [ ]:
df[df["is_anomaly"]].head(10)

,user_id,timestamp,file_accessed,access_type,location,role,failed_logins,hour,day_of_week,anomaly_failed_login,anomaly_off_hours,anomaly_excessive_access,is_anomaly
1,1034,2024-01-01 09:05:00,Finance_Report,read,Office,Employee,1,9,Monday,False,False,True,True
3,1103,2024-01-01 07:21:00,Client_Data,read,Remote,Admin,0,7,Monday,False,True,False,True
8,1035,2024-01-01 07:38:00,Finance_Report,delete,Office,Employee,0,7,Monday,False,True,False,True
10,1011,2024-01-01 09:56:00,Engineering_Design,write,Office,Employee,0,9,Monday,False,False,True,True
11,1001,2024-01-01 08:09:00,Engineering_Design,read,Office,Employee,0,8,Monday,False,False,True,True
18,1067,2024-01-01 06:32:00,Finance_Report,read,Remote,Employee,0,6,Monday,False,True,False,True
22,1057,2024-01-01 09:56:00,HR_Records,read,Office,Employee,11,9,Monday,True,False,False,True
24,1069,2024-01-01 09:42:00,Engineering_Design,read,Office,Employee,0,9,Monday,False,False,True,True
25,1069,2024-01-01 07:23:00,HR_Records,delete,Office,Employee,0,7,Monday,False,True,True,True
27,1069,2024-01-01 11:12:00,Engineering_Design,read,Office,Employee,0,11,Monday,False,False,True,True


In [ ]:
df[["anomaly_failed_login", "anomaly_off_hours", "anomaly_excessive_access"]].sum()

anomaly_failed_login         750
anomaly_off_hours           2393
anomaly_excessive_access    4376
dtype: int64

In [ ]:
df.to_csv("../data/access_logs_with_anomalies.csv", index=False)
print("Anomaly-tagged dataset saved.")

Anomaly-tagged dataset saved.
